# Project 3 - Graph Algorithm Implementation

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering


# Implementation of Betweeness Centrality Graph Algorithm

Betweeness Centrality compute all pairs shortest path (weighted). For each node, compute how many paths pass through the node. The pivotal node lies on every path between two other nodes. In this implemenation, we would place a backup delivery struck at the stations with the highest betweeness scores so that there a fail safe system in case that station is down for the day. 

In [102]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [103]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))

In [104]:
session = driver.session(database="neo4j")

In [105]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [106]:
# since the BART systems looked connected,
# we will be using gds page rank algorithm to rank all of the given station and output it in a tabular format
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

In [112]:
query = """

CALL gds.betweenness.stream('ds_graph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as betweenness
ORDER BY betweenness DESC

"""

df = my_neo4j_run_query_pandas(query)
df = df[df['betweenness'] > 0]
df['name'] = [" ".join(n[1:]) for n in df['name'].str.split(" ")] # remove the line
df = df.groupby('name').mean().reset_index().sort_values('betweenness', ascending=False)
df.head(10)

,name,betweenness
39,Rockridge,5509.000000
33,Orinda,4997.000000
24,Lafayette,4469.000000
0,12th Street,4176.426217
26,MacArthur,4138.430736
25,Lake Merritt,3988.568790
46,Walnut Creek,3925.000000
2,19th Street,3888.013767
21,Fruitvale,3697.419834
49,West Oakland,3476.772638
